# Plot variants escape scores

In [ ]:
import altair as alt

import altair_saver

import bindingcalculator

import pandas as pd

Read in data:

In [ ]:
bindcalc = bindingcalculator.BindingCalculator()

data = (
    pd.read_csv(snakemake.input.csv, na_filter=False)
    .assign(mut_list=lambda x: x['mutated RBD sites'].map(lambda s:
                                                          [int(m) for m in s.split()]),
            escape_score=lambda x: x['mut_list'].map(bindcalc.binding_retained),
            )
    .rename(columns={'escape_score': 'calculated binding score'})
    )

data

Plot data:

In [ ]:
chart = (
    alt.Chart(data)
    .encode(y=alt.Y('variant',
                    sort=data.sort_values('calculated binding score', ascending=False)['variant'].tolist(),
                    ),
            x=alt.X('calculated binding score',
                    scale=alt.Scale(type='log',
                                    nice=False,
                                    domain=(0.85 * data['calculated binding score'].min(),
                                            data['calculated binding score'].max()),
                                    ),
                    ),
            tooltip=['variant', 'mutated RBD sites', alt.Tooltip('calculated binding score', format='.2f')],
            )
    .mark_point(filled=True, size=70)
    .properties(width=320, height=15 * data['variant'].nunique())
    )

chart.save(snakemake.output.html)

altair_saver.save(chart, snakemake.output.png, vega_cli_options=['-s 4'])

chart

Write information variants to LaTex table:

In [ ]:
with pd.option_context("max_colwidth", 1000):
    (data
     [['variant', 'mutated RBD sites']]
     .to_latex(snakemake.output.table, index=False)
     )